In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns

In [5]:
pd.__version__

'0.25.3'

In [6]:
np.__version__

'1.18.1'

In [2]:
sns.__version__

'0.9.0'

# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas Series and DataFrame objects, respectively. Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys. While Pandas does provide Panel and Panel4D objects that natively handle three-dimensional and four-dimensional data (see Aside: Panel Data), a far more common pattern in practice is to make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index. In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In this section, we'll explore the direct creation of MultiIndex objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

# A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional Series. For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way

Suppose you would like to track data about states from two different years. Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [18]:
index=[('California',2000),('California',2010),
    ('New York',2000),('New York',2010),
    ('Texas',2000),('Texas',2010)]
populations = [33871648,37253956,
              18976457,19378102,
              20851820,2514561]

In [19]:
pop = pd.Series(populations, index=ind)

In [5]:
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)          2514561
dtype: int64

In [8]:
pop[0:5]

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [10]:
pop[('California',2010):('Texas',2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [11]:
pop[[i for i in pop.index if i[1]==2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)          2514561
dtype: int64

This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

In [19]:
for i in pop.index:
    if i[1]==2010:
        print(i,pop[i])

('California', 2010) 37253956
('New York', 2010) 19378102
('Texas', 2010) 2514561


In [15]:
pop.index

Index([('California', 2000), ('California', 2010),   ('New York', 2000),
         ('New York', 2010),      ('Texas', 2000),      ('Texas', 2010)],
      dtype='object')

### The Better Way: Pandas MultiIndex

Fortunately, Pandas provides a better way. Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas MultiIndex type gives us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

Better way to perform indexing

In [21]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [23]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010     2514561
dtype: int64

In [31]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

In [32]:
pop[2]

18976457

In [33]:
pop['California'][2010]

37253956

In [38]:
pop[::-1]

Texas       2010     2514561
            2000    20851820
New York    2010    19378102
            2000    18976457
California  2010    37253956
            2000    33871648
dtype: int64

In [39]:
pop[:,2010]

California    37253956
New York      19378102
Texas          2514561
dtype: int64

In [22]:
pop.ndim

1

### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and column labels. In fact, Pandas is built with this equivalence in mind. The unstack() method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:

In [42]:
pop_df =pop.unstack() #unstack() is 
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,2514561


In [43]:
pop_df.ndim

2

In [44]:
se = pop_df.stack()
se

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010     2514561
dtype: int64

In [45]:
pop_df = pd.DataFrame({'Total':pop,
                      'Under18':[9267089,9284094,
                                4687374,4318033,
                                5906301,6879014]})
pop_df

Total  Under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010   2514561  6879014

In [46]:
pop_df['Total']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010     2514561
Name: Total, dtype: int64

In [47]:
pop_df['Under18']

California  2000    9267089
            2010    9284094
New York    2000    4687374
            2010    4318033
Texas       2000    5906301
            2010    6879014
Name: Under18, dtype: int64

In [57]:
f_u18 =pop_df['Under18']/pop_df['Total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,2.735672


# Methods of MultiIndex Creation

In [48]:
df = pd.DataFrame(np.random.rand(4,2),
                 index=[['a','a','b','b'],[1,2,1,2]],
                 columns = ['data1','data2'])

In [49]:
df

data1     data2
a 1  0.700499  0.373326
  2  0.462014  0.602927
b 1  0.734093  0.622153
  2  0.856127  0.582514

In [50]:
df1 =pd.DataFrame(np.random.rand(4,2),
                 index=[['a','b','d','c'],[1,2,1,2]],
                 columns = ['data1','data2'])

In [51]:
df1

,,data1,data2
a,1,0.068901,0.836388
b,2,0.973402,0.342602
d,1,0.051201,0.114523
c,2,0.803599,0.387532


In [53]:
df2=pd.DataFrame(np.random.rand(4,2),
                 index=[['a','a','a','b'],[1,2,1,2]],
                 columns = ['data1','data2'])
df2

data1     data2
a 1  0.054578  0.007770
  2  0.044741  0.279654
  1  0.219497  0.149598
b 2  0.174557  0.235322

In [56]:
data = {('California', 2000):33871648,
        ('California', 2010):37253956,
        ('New York', 2000):18976457,
        ('New York', 2010):19378102,
        ('Texas', 2000):20851820,
        ('Texas', 2010):2514561}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010     2514561
dtype: int64

In [58]:
pd.MultiIndex.from_arrays([['a','a','b','b'],[1,2,1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [59]:
pd.MultiIndex.from_tuples([('a',1),('a',2),('b',1),('b',2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [60]:
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [67]:
pd.MultiIndex(levels=[['a','b'],[1,2]],
             codes=[[0,0,1,1],[0,1,0,1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

# MultiIndex level names

In [66]:
pop.index.names=['state','year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010     2514561
dtype: int64

# MultiIndex level columns

In [24]:
#hierarchical index for rows and columns
index=pd.MultiIndex.from_product([[2013,2014],[1,2,3]],
                                names=['year','visit'])
columns = pd.MultiIndex.from_product([['Rani','Raju','Sam'],['BMI','Temp','Weight']],
                                    names=['Name','type'])

In [25]:
# mock some data
data = np.round(np.random.randn(6,9),2)
data

array([[ 1.28,  0.57, -1.35,  0.06, -1.17, -0.9 , -1.68, -0.85,  1.3 ],
       [ 1.19,  0.18, -1.41,  0.93, -0.22, -0.95,  0.62, -0.27, -0.14],
       [-1.94,  0.99, -0.18, -0.06, -0.29,  1.28,  0.99,  0.46, -0.66],
       [ 0.74,  1.09,  0.9 , -0.92, -0.23,  0.8 , -0.49,  0.64,  1.05],
       [ 2.46,  1.8 ,  0.48, -0.42, -0.92, -0.17,  1.92, -0.2 , -0.96],
       [ 0.28,  0.68, -0.61,  1.04,  0.76,  0.94,  1.05, -0.76,  0.71]])

In [26]:
data[:, ::2]*=10
data

array([[ 12.8 ,   0.57, -13.5 ,   0.06, -11.7 ,  -0.9 , -16.8 ,  -0.85,
         13.  ],
       [ 11.9 ,   0.18, -14.1 ,   0.93,  -2.2 ,  -0.95,   6.2 ,  -0.27,
         -1.4 ],
       [-19.4 ,   0.99,  -1.8 ,  -0.06,  -2.9 ,   1.28,   9.9 ,   0.46,
         -6.6 ],
       [  7.4 ,   1.09,   9.  ,  -0.92,  -2.3 ,   0.8 ,  -4.9 ,   0.64,
         10.5 ],
       [ 24.6 ,   1.8 ,   4.8 ,  -0.42,  -9.2 ,  -0.17,  19.2 ,  -0.2 ,
         -9.6 ],
       [  2.8 ,   0.68,  -6.1 ,   1.04,   7.6 ,   0.94,  10.5 ,  -0.76,
          7.1 ]])

In [27]:
data+=37
data

array([[49.8 , 37.57, 23.5 , 37.06, 25.3 , 36.1 , 20.2 , 36.15, 50.  ],
       [48.9 , 37.18, 22.9 , 37.93, 34.8 , 36.05, 43.2 , 36.73, 35.6 ],
       [17.6 , 37.99, 35.2 , 36.94, 34.1 , 38.28, 46.9 , 37.46, 30.4 ],
       [44.4 , 38.09, 46.  , 36.08, 34.7 , 37.8 , 32.1 , 37.64, 47.5 ],
       [61.6 , 38.8 , 41.8 , 36.58, 27.8 , 36.83, 56.2 , 36.8 , 27.4 ],
       [39.8 , 37.68, 30.9 , 38.04, 44.6 , 37.94, 47.5 , 36.24, 44.1 ]])

In [28]:
# create the DataFrame
health_data = pd.DataFrame(data,index=index,columns=columns)
health_data

Name        Rani                 Raju                Sam              
type         BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 1      49.8  37.57   23.5  37.06  25.3  36.10  20.2  36.15   50.0
     2      48.9  37.18   22.9  37.93  34.8  36.05  43.2  36.73   35.6
     3      17.6  37.99   35.2  36.94  34.1  38.28  46.9  37.46   30.4
2014 1      44.4  38.09   46.0  36.08  34.7  37.80  32.1  37.64   47.5
     2      61.6  38.80   41.8  36.58  27.8  36.83  56.2  36.80   27.4
     3      39.8  37.68   30.9  38.04  44.6  37.94  47.5  36.24   44.1

In [29]:
health_data.to_csv("health_data_for_multiindexing.csv")

In [30]:
health_data['Rani']

type         BMI   Temp  Weight
year visit                     
2013 1      49.8  37.57    23.5
     2      48.9  37.18    22.9
     3      17.6  37.99    35.2
2014 1      44.4  38.09    46.0
     2      61.6  38.80    41.8
     3      39.8  37.68    30.9

In [31]:
health_data.iloc[1]

Name  type  
Rani  BMI       48.90
      Temp      37.18
      Weight    22.90
Raju  BMI       37.93
      Temp      34.80
      Weight    36.05
Sam   BMI       43.20
      Temp      36.73
      Weight    35.60
Name: (2013, 2), dtype: float64

In [32]:
health_data.iloc[1:2]

Name        Rani                 Raju                Sam              
type         BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 2      48.9  37.18   22.9  37.93  34.8  36.05  43.2  36.73   35.6

In [33]:
health_data.iloc[0:5]

Name        Rani                 Raju                Sam              
type         BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 1      49.8  37.57   23.5  37.06  25.3  36.10  20.2  36.15   50.0
     2      48.9  37.18   22.9  37.93  34.8  36.05  43.2  36.73   35.6
     3      17.6  37.99   35.2  36.94  34.1  38.28  46.9  37.46   30.4
2014 1      44.4  38.09   46.0  36.08  34.7  37.80  32.1  37.64   47.5
     2      61.6  38.80   41.8  36.58  27.8  36.83  56.2  36.80   27.4

In [36]:
health_data['Rani']

type         BMI   Temp  Weight
year visit                     
2013 1      49.8  37.57    23.5
     2      48.9  37.18    22.9
     3      17.6  37.99    35.2
2014 1      44.4  38.09    46.0
     2      61.6  38.80    41.8
     3      39.8  37.68    30.9

In [37]:
health_data['Sam']

type         BMI   Temp  Weight
year visit                     
2013 1      20.2  36.15    50.0
     2      43.2  36.73    35.6
     3      46.9  37.46    30.4
2014 1      32.1  37.64    47.5
     2      56.2  36.80    27.4
     3      47.5  36.24    44.1

In [41]:
health_data.loc[:, ('Raju')]

type          BMI  Temp  Weight
year visit                     
2013 1      37.06  25.3   36.10
     2      37.93  34.8   36.05
     3      36.94  34.1   38.28
2014 1      36.08  34.7   37.80
     2      36.58  27.8   36.83
     3      38.04  44.6   37.94